In [1]:
# pip install openpyxl

In [2]:
import pandas as pd
import os
# from itertools import combinations
# import openpyxl

In [3]:
file_path = "C:/Users/José Roberto/Documents/Baygon Geral/Codigo/associacao-futebol/Teste_1_Jogador_Completo/"
file_name = "Sumula_Novo_Padrao.xlsx"
full_file_path = file_path+file_name

sumula = pd.read_excel(
    full_file_path,
    sheet_name=None,
    header=1,
    engine="calamine"
)

In [4]:
sumula_individual = sumula["Individual"][sumula["Individual"]["Boleiro"].notna()]

sumula_individual[["Gols", "Assistencias", "Vitorias", "Derrotas", "Empates", "Presenca",
                      "Pontos no Baba", "Mensalidade", "Colaboracao", "Pontos de Torneio", "Pote"]] = \
    sumula_individual[["Gols", "Assistencias", "Vitorias", "Derrotas", "Empates", "Presenca",
                          "Pontos no Baba", "Mensalidade", "Colaboracao", "Pontos de Torneio", "Pote"]].fillna(0).astype(int)
# sumula_individual

C:\Users\José Roberto\AppData\Local\Temp\ipykernel_19616\3773807634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sumula_individual[["Gols", "Assistencias", "Vitorias", "Derrotas", "Empates", "Presenca",


In [5]:
# (OK) Pizza deve fazer parte da aba Times, e não ser uma aba à parte (simplifica absurdamente o código)

# lista_resultados_pizza = []
# sumula["Pizza"]["Pizza"] = 1
# for col in sumula["Pizza"].columns:
#     if col.startswith("Colete"):
#         resultados_pizza = sumula["Individual"].merge(sumula["Pizza"], how="left", left_on=["Baba", "Colete"], right_on=["Baba", col])
#         lista_resultados_pizza.append(resultados_pizza)
# sumula_pizza = pd.concat(lista_resultados_pizza, ignore_index=True)
# sumula_pizza["Pizza"] = sumula_pizza["Pizza"].fillna(0).astype(int)

# sumula_pizza = sumula_pizza.drop(
#     columns=[col for col in sumula_pizza.columns if col.startswith("Colete") and col != "Colete"]
# )

# # Quando há mais de um "colete" vencedor, os resultados são duplicados, sendo necessário remover as duplicatas
# sumula_pizza = sumula_pizza.sort_values(by="Pizza", ascending=False).drop_duplicates(subset=["Baba", "Boleiro"], keep="first")

# sumula_pizza = sumula_pizza.sort_values(by=["Baba", "Colete", "Boleiro"], ascending=True).reset_index(drop=True)

In [6]:
# sumula_defesaspenalti = sumula_pizza.merge(sumula["DefesasPenaltis"], how="left", on=["Baba", "Boleiro"])
# sumula_defesaspenalti = sumula["Individual"]
# sumula_defesaspenalti["Defesas de Penalti"] = sumula_defesaspenalti["Defesas de Penalti"].fillna(0).astype("int")
# sumula_defesaspenalti

In [7]:
# Ouros (O), Gols Contra (C), Cartões (C), Pizza (P)
sumula_agregada = (sumula_individual
                .merge(sumula["DefesasPenaltis"], how="left", on=["Baba", "Boleiro"])
                .merge(sumula["Ouros"], how="left", on=["Baba", "Boleiro"])
                .merge(sumula["Contra"], how="left", on=["Baba", "Boleiro"])
                .merge(sumula["PontosCopa"], how="left", on=["Baba", "Boleiro"])
                .merge(sumula["Cartoes"], how="left", on=["Baba", "Boleiro"])
                .merge(sumula["Times"][["Baba", "Colete", "Pizza"]], how="left", on=["Baba", "Colete"])
                .merge(sumula["Filmagens"], how="left", on=["Baba", "Boleiro"])
              )

# Devido a uma partida ter sido filmada por duas pessoas, o campo "Filmagens" passará a ter casa decimal (0,5 para cada)
sumula_agregada["Filmagens"] = sumula_agregada["Filmagens"].fillna(0).astype(float).round(1)
sumula_agregada[["Defesas de Penalti", "Ouros", "Gols Contra", "Pontos de Copa", "Cartao Amarelo", 
                 "Cartao Azul", "Cartao Vermelho", "Pizza"]] = \
          sumula_agregada[["Defesas de Penalti", "Ouros", "Gols Contra", "Pontos de Copa", 
                 "Cartao Amarelo", "Cartao Azul", "Cartao Vermelho", "Pizza"]].fillna(0).astype(int)
# sumula_agregada

C:\Users\José Roberto\AppData\Local\Temp\ipykernel_19616\1451030371.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  "Cartao Amarelo", "Cartao Azul", "Cartao Vermelho", "Pizza"]].fillna(0).astype(int)


In [8]:
df_premios = sumula["Premios"].melt(id_vars=["Baba", "Premio"], value_name="Boleiro")[["Baba", "Premio", "Boleiro"]]
df_premios = df_premios.dropna(subset=["Boleiro"]).reset_index(drop=True)
df_premios["Valor"] = 1
df_premios = df_premios.pivot_table(index=["Baba", "Boleiro"], columns="Premio", values="Valor", fill_value=0)

df_premios.columns = [f"Premio {col}" if col in ["Gol", "Drible", "Defesa",
                                                 "Inacreditavel"] else col for col in df_premios.columns]
df_premios = df_premios.reset_index()

sumula_premios = sumula_agregada.merge(df_premios, how="left", left_on=["Baba", "Boleiro"], right_on=["Baba", "Boleiro"])
sumula_premios = sumula_premios.fillna(0).astype({"Pote":int, "Premio Gol":int, "Premio Drible":int, "Premio Defesa":int,
                                          "Premio Inacreditavel":int, "Bola Cheia":int, "Bola Murcha":int,
                                          "Clean Sheets":int, "Roleta":int, "Primeiro Gol":int, "Primeira Assistencia":int})
# sumula_premios

In [9]:
# Presença será preenchida manualmente, para abranger casos de boleiros que receberam alguma pontuação sem jogar

# sumula_presenca = sumula_premios.copy()
# sumula_presenca["Presenca"] = 1
# sumula_presenca

In [10]:
# df_primeiro_gol = sumula["PrimeiroGol"].copy()
# df_primeiro_gol["Primeiro Gol"] = 1
# sumula_primeiro_gol = sumula_presenca.merge(df_primeiro_gol, on=["Baba", "Boleiro"], how="left")
# sumula_primeiro_gol["Primeiro Gol"] = sumula_primeiro_gol["Primeiro Gol"].fillna(0).astype(int)
# # display(sumula_primeiro_gol)

# # df_primeiro_gol_agrupado = df_primeiro_gol.groupby("Boleiro", as_index=False).count().copy()
# df_primeiro_gol_agrupado = df_primeiro_gol.groupby("Boleiro")["Baba"].count().reset_index(name="Primeiro Gol").copy()
# # df_primeiro_gol_agrupado

In [11]:
# sumula_ranking = sumula_primeiro_gol.copy()
sumula_ranking = sumula_premios.copy()
sumula_ranking["Ranking Geral"] = (
    sumula_ranking["Gols"]
    + sumula_ranking["Assistencias"]
    + sumula_ranking["Vitorias"] * 3
    + sumula_ranking["Empates"] * 1 # Estava errado, multiplicando por 3
    + sumula_ranking["Mensalidade"]
    + sumula_ranking["Colaboracao"]
    + sumula_ranking["Defesas de Penalti"]
    + sumula_ranking["Bola Cheia"] * 3
    + sumula_ranking["Clean Sheets"] * 3
    + sumula_ranking["Premio Defesa"] * 3
    + sumula_ranking["Premio Drible"] * 3
    + sumula_ranking["Premio Gol"] * 3
    + sumula_ranking["Presenca"]
    + sumula_ranking["Pontos de Torneio"]
    + sumula_ranking["Pontos de Copa"] # Esqueci; estava subtraindo de "Ranking Selecao" sem somar aqui
)

sumula_ranking["Ranking Selecao"] = sumula_ranking["Ranking Geral"] - sumula_ranking["Colaboracao"] - sumula_ranking["Mensalidade"] \
                                        - sumula_ranking["Pontos de Copa"]
sumula_ranking["Ranking Selecao"] = sumula_ranking["Ranking Selecao"].astype(int)
df_final_sumula_ranking = sumula_ranking.copy()
# df_final_sumula_ranking

In [12]:
# # Verificação de Pontuação do Mês
#
# sumula_ranking[
#     (sumula_ranking["Boleiro"] == "Leitinho")
# ][[
#     "Baba", "Boleiro",
#     "Gols", "Assistencias", "Vitorias", "Empates", "Defesas de Penalti",
#     "Bola Cheia", "Clean Sheets", "Premio Defesa", "Premio Drible",
#     "Premio Gol", "Presenca", "Pontos de Torneio"
# ]]

In [13]:
ordem_posicoes = ["Goleiro", "Zagueiro", "Meio", "Ala", "Atacante"]
ordem_meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
    "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]

In [14]:
df_boleiros = sumula["Boleiros"].copy()
df_baba = sumula["Baba"][["Baba", "Data"]].copy()

df_baba["Mes"] = df_baba["Data"].dt.month_name(locale="Portuguese_Brazil")
df_baba["Ano"] = df_baba["Data"].dt.year

df_validacao_ultimo_baba_mes = df_baba.copy()
df_validacao_ultimo_baba_mes = df_validacao_ultimo_baba_mes.sort_values(
    ["Ano", "Mes", "Baba"], ascending=[True, True, False]
)

df_validacao_ultimo_baba_mes["Total Babas"] = (
    df_validacao_ultimo_baba_mes.groupby(["Ano", "Mes"])["Baba"].transform("count")
)

df_validacao_ultimo_baba_mes = df_validacao_ultimo_baba_mes.drop_duplicates(subset=["Ano", "Mes"])[
    ["Ano", "Mes", "Baba", "Total Babas"]
]
df_validacao_ultimo_baba_mes = df_validacao_ultimo_baba_mes[["Baba", "Total Babas"]].reset_index(drop=True)


df_ultimo_baba_mes = sumula_ranking.merge(df_baba, on=["Baba"], how="left")
df_ultimo_baba_mes = df_ultimo_baba_mes.sort_values(
    ["Ano", "Mes", "Baba"], ascending=[True, True, False]
)
df_ultimo_baba_mes = df_ultimo_baba_mes.drop_duplicates(subset=["Ano", "Mes"])[
    ["Ano", "Mes", "Baba"]
]

df_ultimo_baba_mes = df_ultimo_baba_mes.rename(columns={"Baba":"Ultimo Baba"}).astype({"Ano":int})

df_ranking_sem_colab = sumula_ranking[["Baba", "Boleiro", "Ranking Selecao", "Vitorias", "Gols", "Assistencias"]].copy()
df_pontuacao_por_mes = df_ranking_sem_colab \
                    .merge(df_boleiros, on="Boleiro", how="left") \
                    .merge(df_baba, on=["Baba"], how="left")
df_pontuacao_por_mes["Mes"] = df_pontuacao_por_mes["Data"].dt.month_name(locale="Portuguese_Brazil")
df_pontuacao_por_mes["Ano"] = df_pontuacao_por_mes["Data"].dt.year
df_pontuacao_por_mes = df_pontuacao_por_mes.groupby(["Ano", "Mes", "Boleiro", "Posicao"])[
    ["Ranking Selecao", "Vitorias", "Gols", "Assistencias"]
].sum().reset_index()
df_pontuacao_por_mes = df_pontuacao_por_mes.merge(df_ultimo_baba_mes, on=["Ano", "Mes"], how="left")

df_pontuacao_por_mes["Posicao"] = pd.Categorical(df_pontuacao_por_mes["Posicao"], categories=ordem_posicoes, ordered=True)
df_pontuacao_por_mes["Mes"] = pd.Categorical(df_pontuacao_por_mes["Mes"], categories=ordem_meses, ordered=True)

df_pontuacao_por_mes = df_pontuacao_por_mes.sort_values(["Ano", "Mes", "Posicao", "Ranking Selecao",
                                                       "Vitorias", "Gols", "Assistencias", "Boleiro"],
                                                       ascending=[False, False, True, False, 
                                                                  False, False, False, True])

# A validação deve acontecer somente na Seleção do Mês. Pontuação do Mês deve conter tudo
# df_pontuacao_por_mes = df_pontuacao_por_mes[
#     df_pontuacao_por_mes["Ultimo Baba"].isin(df_validacao_ultimo_baba_mes["Baba"])
# ]

df_pontuacao_por_mes = df_pontuacao_por_mes \
    .merge(df_validacao_ultimo_baba_mes, left_on="Ultimo Baba", right_on="Baba", how="left") \
    .drop(columns="Baba")

# df_pontuacao_por_mes

In [15]:
df_selecao_do_mes = (
    df_pontuacao_por_mes
    [df_pontuacao_por_mes["Total Babas"] >= 3]
    .sort_values(
        by=["Ano", "Mes", "Posicao", "Ranking Selecao", "Vitorias", "Gols", "Assistencias", "Boleiro"],
        ascending=[True, False, True, False, False, False, False, True]
    )
    .groupby(["Ano", "Mes", "Posicao"], as_index=False, observed=True)
    .head(1)
    .reset_index(drop=True)
    .copy()
)

df_selecao_do_mes = df_selecao_do_mes[
    df_selecao_do_mes["Ultimo Baba"].isin(df_validacao_ultimo_baba_mes["Baba"])
]

df_selecao_do_mes = df_selecao_do_mes \
    .merge(df_validacao_ultimo_baba_mes, left_on="Ultimo Baba", right_on="Baba", how="left") \
    .drop(columns="Baba")

remover_colunas = ["Vitorias", "Gols", "Assistencias"]
# ordenar_selecao_do_mes = ["Ano", "Mes", "Ultimo Baba", "Boleiro", "Posicao", "Pontuacao"]
ordenar_selecao_do_mes = ["Ano", "Mes", "Boleiro", "Posicao", "Pontuacao"]

df_selecao_do_mes = df_selecao_do_mes.rename(columns={"Ranking Selecao":"Pontuacao"}).drop(columns=remover_colunas)
df_final_selecao_do_mes = df_selecao_do_mes[ordenar_selecao_do_mes]

# df_final_selecao_do_mes

In [16]:
df_final_pontuacao_por_mes = df_pontuacao_por_mes.rename(columns={"Ranking Selecao": "Pontuacao"}).drop(columns=["Ultimo Baba", "Total Babas"])
# df_final_pontuacao_por_mes

In [17]:
# df_ultima_pizza = sumula_ranking[["Baba", "Boleiro", "Pizza"]].sort_values("Baba", ascending=False).copy()
# lista_ultima_pizza = df_ultima_pizza[df_ultima_pizza["Pizza"] == 1].groupby("Boleiro")["Baba"].first()
# df_ultima_pizza["Ultima Pizza"] = df_ultima_pizza["Boleiro"].map(lista_ultima_pizza).fillna("Não ganhou")
# df_ultima_pizza = df_ultima_pizza[["Boleiro", "Ultima Pizza"]].drop_duplicates().reset_index(drop=True)
# df_ultima_pizza

In [18]:
df_ultima_presenca = (
    sumula_ranking[["Baba", "Boleiro"]]
    .sort_values("Baba", ascending=False)
    .groupby("Boleiro", as_index=False)["Baba"]
    .first()
    .rename(columns={"Baba": "Ultima Presenca"})
    .copy()
)
# df_ultima_presenca

In [19]:
colunas_para_somar = [
    "Ranking Geral", "Gols", "Assistencias", "Vitorias", "Derrotas", "Empates", "Pontos no Baba",
    "Mensalidade", "Colaboracao", "Pontos de Torneio", "Presenca", "Pizza", "Defesas de Penalti",
    "Ouros", "Gols Contra", "Pontos de Copa", "Cartao Amarelo", "Cartao Azul", "Cartao Vermelho",
    "Bola Cheia", "Bola Murcha", "Clean Sheets", "Premio Defesa", "Premio Drible",
    "Premio Gol", "Premio Inacreditavel", "Roleta", "Primeiro Gol", "Primeira Assistencia", "Filmagens" #, "Ranking Selecao"
]

df_ranking_agrupado = (
    sumula_ranking
    .groupby("Boleiro", as_index=False)[colunas_para_somar]
    .sum()
    .copy()
)

df_ranking_completo = (df_ranking_agrupado
                    #    .merge(df_ultima_pizza, on="Boleiro", how="left")
                       .merge(df_ultima_presenca, on="Boleiro", how="left")
                    #    .merge(df_primeiro_gol_agrupado, on="Boleiro", how="left")
                       .merge(df_boleiros, on="Boleiro", how="left")
                       )

# df_ranking_completo["Primeiro Gol"] = df_ranking_completo["Primeiro Gol"].fillna(0).astype(int)

df_final_ranking_completo = df_ranking_completo
# df_final_ranking_completo

In [20]:
df_boleiros_times = sumula["Individual"][["Baba", "Colete", "Boleiro"]].dropna(subset="Baba")
df_times = sumula["Times"].dropna(subset="Baba")

df_duplas = df_boleiros_times.merge(df_times, on=["Baba", "Colete"], how="left")
df_final_duplas = (
    df_duplas[["Baba", "Colete", "Boleiro"]].merge(df_duplas, on=["Baba", "Colete"])
    .query("Boleiro_x != Boleiro_y")
    .rename(columns={"Boleiro_x": "Boleiro", "Boleiro_y": "Boleiro2"})
    .reset_index(drop=True)
)
df_final_duplas[["Vitorias", "Derrotas", "Empates", "Pizza"]] = \
    df_final_duplas[["Vitorias", "Derrotas", "Empates", "Pizza"]].astype(int)
# df_final_duplas

# MemoryError: Unable to allocate 8.00 TiB for an array with shape (1099293535300,) and data type int64 #####

In [21]:
# Solução ChatGPT para erro de uso de memória do Pandas ###

# df_duplas_com_times = sumula["Individual"][["Baba", "Colete", "Boleiro"]].merge(
#     sumula["Times"],
#     on=["Baba", "Colete"],
#     how="left"
# )

# duplas = []

# for (baba, colete), group in df_duplas_com_times.groupby(["Baba", "Colete"]):
#     boleiros = group["Boleiro"].dropna().unique()
#     stats_row = group.iloc[0]
    
#     for b1, b2 in combinations(boleiros, 2):
#         for boleiro_a, boleiro_b in [(b1, b2), (b2, b1)]:
#             duplas.append({
#                 "Baba": baba,
#                 "Colete": colete,
#                 "Boleiro": boleiro_a,
#                 "Boleiro2": boleiro_b,
#                 "Vitorias": stats_row["Vitorias"],
#                 "Derrotas": stats_row["Derrotas"],
#                 "Empates": stats_row["Empates"],
#                 "Pizza": stats_row["Pizza"]
#             })

# df_final_duplas = pd.DataFrame(duplas)

In [22]:
grupo_duplas = df_final_duplas.groupby(["Boleiro", "Boleiro2"]).agg(
    Total_Pizza=("Pizza", "sum"),
    Total_Vitorias=("Vitorias", "sum"),
    Total_Derrotas=("Derrotas", "sum"),
    Total_Empates=("Empates", "sum"),
    Total_Babas=("Baba", "count")
    
).reset_index()

grupo_duplas["Total_Partidas"] = grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"] + grupo_duplas["Total_Empates"]
grupo_duplas["P_Vitorias"] = grupo_duplas["Total_Vitorias"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_Derrotas"] = grupo_duplas["Total_Derrotas"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_Empates"] = grupo_duplas["Total_Empates"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_VD"] = grupo_duplas["Total_Vitorias"] / (grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"])
grupo_duplas["M_Pizza"] = grupo_duplas["Total_Pizza"] / grupo_duplas["Total_Babas"]
grupo_duplas["M_Vitorias"] = grupo_duplas["Total_Vitorias"] / grupo_duplas["Total_Babas"]
grupo_duplas["M_Derrotas"] = grupo_duplas["Total_Derrotas"] / grupo_duplas["Total_Babas"]
grupo_duplas["M_Empates"] = grupo_duplas["Total_Empates"] / grupo_duplas["Total_Babas"]
grupo_duplas = grupo_duplas[
            [
                "Boleiro",
                "Boleiro2",
                "Total_Pizza",
                "Total_Vitorias",
                "Total_Derrotas",
                "Total_Empates",
                "Total_Partidas",
                "Total_Babas",
                "P_Vitorias",
                "P_Derrotas",
                "P_Empates",
                "P_VD",
                "M_Pizza",
                "M_Vitorias",
                "M_Derrotas",
                "M_Empates"
            ]
        ]

grupo_duplas.columns = grupo_duplas.columns.str.replace("_", " ")
grupo_duplas[["Total Pizza", "Total Vitorias", "Total Derrotas", "Total Empates", "Total Partidas"]] = \
            grupo_duplas[["Total Pizza", "Total Vitorias", "Total Derrotas", "Total Empates", "Total Partidas"]].astype(int)

df_final_grupo_duplas = grupo_duplas.sort_values(["Total Babas", "Total Partidas", "Total Vitorias"], 
                                                 ascending=False).reset_index(drop=True)
# df_final_grupo_duplas

In [23]:
df_numeros_baba = df_final_sumula_ranking.groupby("Baba", as_index=False).sum().copy()
df_numeros_baba = df_numeros_baba[["Baba", "Gols", "Assistencias", "Defesas de Penalti", "Ouros", "Gols Contra",
                                   "Cartao Amarelo", "Cartao Azul", "Cartao Vermelho", "Bola Cheia", "Bola Murcha",
                                   "Clean Sheets", "Premio Defesa", "Premio Drible", "Premio Gol", "Premio Inacreditavel",
                                   "Roleta", "Presenca", "Filmagens"]]
# pd.set_option("display.max_columns", None)
# display(df_numeros_baba)

df_partidas_baba = sumula["Times"].groupby("Baba", as_index=False).sum().copy()
# df_partidas_baba["Empates"] = df_partidas_baba["Empates"] + 1 ############ Teste de divergência
# df_partidas_baba["Vitorias"] = df_partidas_baba["Vitorias"] + 1 ############ Teste de divergência


log_file_path = "log_divergencias_partidas.txt"

# Validação de quantidade de partidas com base em vitórias, derrotas e empates
df_diff_resultados = df_partidas_baba[(df_partidas_baba["Vitorias"] != df_partidas_baba["Derrotas"]) |
    (df_partidas_baba["Empates"] % 2 == 1)][["Baba", "Vitorias", "Derrotas", "Empates"]]

if not df_diff_resultados.empty:
    with open(log_file_path, "w", encoding="utf-8") as f:
        f.write("Erros no registro dos resultados de partidas: \n"
        "Verificação: Vitórias e Derrotas devem ser iguais, e o número de Empates deve ser par. \n")
        for _, row in df_diff_resultados.iterrows():
            if row["Vitorias"] != row["Derrotas"]:
                f.write(
                    f"- Baba: {row['Baba']} | Vitorias: {row['Vitorias']} | Derrotas: {row['Derrotas']}\n"
                )
            if row["Empates"] % 2 == 1:
                f.write(
                    f"- Baba: {row['Baba']} | Empates: {row['Empates']}\n"
                )

df_partidas_baba["Partidas"] = (df_partidas_baba["Vitorias"] + df_partidas_baba["Derrotas"] + df_partidas_baba["Empates"]) / 2
df_partidas_baba["Partidas"] = df_partidas_baba["Partidas"].astype(int)
# display(df_partidas_baba)
df_partidas_baba = df_partidas_baba.rename(columns={"Vitorias":"Com Vencedor", "Empates":"Empatadas"})
df_partidas_baba["Empatadas"] = df_partidas_baba["Empatadas"] / 2 # Total de partidas empatadas é a metade do que os times empataram
df_partidas_baba["Empatadas"] = df_partidas_baba["Empatadas"].astype(int)
df_partidas_baba = df_partidas_baba[["Baba", "Partidas", "Com Vencedor", "Empatadas", "Pizza"]]
# display(df_partidas_baba)

df_filmagens_baba = sumula["Filmagens"].groupby("Baba", as_index=False).sum()[["Baba", "Filmagens"]].copy()
# display(df_filmagens_baba)

df_informacoes_baba = df_numeros_baba.merge(df_partidas_baba, on="Baba", how="left")

# Validação de quantidade de partidas com base em número de filmagens e de partidas, que devem ser iguais
df_diff_filmagens = df_informacoes_baba[(df_informacoes_baba["Partidas"] != df_informacoes_baba["Filmagens"])]
if not df_diff_filmagens.empty:
    if not df_diff_resultados.empty:
        with open(log_file_path, "a", encoding="utf-8") as f:
            f.write("\nErros no registro das filmagens de partidas: \n"
            "Verificação: Partidas e Filmagens devem ser iguais. \n")
            for _, row in df_diff_filmagens.iterrows():
                if row["Partidas"] != row["Filmagens"]:
                    f.write(
                        f"- Baba: {row['Baba']} | Partidas: {row['Partidas']} | Filmagens: {row['Filmagens']}\n"
                    )
    else:
        with open(log_file_path, "w", encoding="utf-8") as f:
            f.write("Erros no registro das filmagens de partidas: \n"
            "Verificação: Partidas e Filmagens devem ser iguais. \n")
            for _, row in df_diff_filmagens.iterrows():
                if row["Partidas"] != row["Filmagens"]:
                    f.write(
                        f"- Baba: {row['Baba']} | Partidas: {row['Partidas']} | Filmagens: {row['Filmagens']}\n"
                    )

# Caso não haja divergências de resultados de partidas ou de filmagens, o arquivo será deletado
if df_diff_resultados.empty and df_diff_filmagens.empty:
    print(f"✅ Quantidades de partidas e filmagens são compatíveis")
    if os.path.exists(log_file_path):
        os.remove(log_file_path)
else:
    print(f"⚠️ Há divergências, consulte {log_file_path}")

# df_informacoes_baba

✅ Quantidades de partidas e filmagens são compatíveis


In [24]:
# writer = pd.ExcelWriter(full_file_path, engine="openpyxl", mode="a", if_sheet_exists="replace")
# with pd.ExcelWriter(full_file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:

# Novo arquivo .xlsx porque incluir abas no antigo leva muito tempo (~48s)
new_file = r"Resultados_Ranking.xlsx"
new_full_file_path = file_path+new_file

def escrever_tabelas_no_excel(writer, dataframe, sheet_name, titulo):
    dataframe.to_excel(writer, sheet_name=sheet_name, startrow=1, index=False)
    worksheet = writer.sheets[sheet_name]
    worksheet.cell(row=1, column=1).value = titulo
    # writer.close() # Substituído pelo "with", que gerencia abertura e fechamento do arquivo, causou erro ao abrir 2ª vez
    
with pd.ExcelWriter(new_full_file_path, engine="openpyxl", mode="w") as writer:
    escrever_tabelas_no_excel(writer, df_final_sumula_ranking, "Sumula por Baba", "Sumula por Baba")
    escrever_tabelas_no_excel(writer, df_final_ranking_completo, "Ranking Completo", "Ranking Completo")
    escrever_tabelas_no_excel(writer, df_final_selecao_do_mes, "Selecao do Mes", "Seleção do Mês")
    escrever_tabelas_no_excel(writer, df_final_pontuacao_por_mes, "Pontuacao por Mes", "Pontuação por Mês")
    escrever_tabelas_no_excel(writer, df_final_grupo_duplas, "Estatisticas por Duplas", "Estatísticas por Duplas")
    # escrever_tabelas_no_excel(writer, df_final_duplas, "Desempenho por Duplas", "Desempenho por Duplas") 